In [10]:
#!pip install pyarrow
#!pip install tensorflow

In [11]:
import numpy as np
import pandas as pd
import pyarrow

In [12]:
df = pd.read_parquet('dataset.parquet', engine='pyarrow')

In [13]:
#Interpolating linearly for the missing frames
df = df.interpolate(method='linear', axis=0, limit_direction='both')

KeyboardInterrupt: 

In [ ]:
df.to_parquet('prep_dataset.parquet', index=True)

In [ ]:
display(df[1828:1831])

In [ ]:
params_columns = ['Participant', 'Exercise', 'Set', 'Camera']
df['SequenceLength'] = df.groupby(params_columns)['time(s)'].transform('count')
max_sequence_length = df['SequenceLength'].max()

padded_sequences = []
for sequence_id, group in df.groupby(params_columns):
    sequence_data = group.iloc[:, 4:-1].values
    pad_width = ((0, max_sequence_length - len(sequence_data)), (0, 0))
    padded_sequence = np.pad(sequence_data, pad_width, mode='constant', constant_values=0)
    padded_sequences.append(padded_sequence)

In [ ]:
padded_df = pd.DataFrame(np.concatenate(padded_sequences, axis=0), columns=df.columns[4:-1])
padded_df[params_columns] = np.repeat(df[params_columns].drop_duplicates().values, max_sequence_length, axis=0)
padded_df = padded_df[['Participant', 'Exercise', 'Set', 'Camera'] + list(padded_df.columns[:-4])]

In [ ]:
print(max_sequence_length)

In [ ]:
#padded_df.to_parquet('padded_dataset.parquet', index=True)